In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action='ignore',category=np.VisibleDeprecationWarning)

## Prediction ##

In [ ]:
families = Table.read_table('family_heights.csv')
families

In [ ]:
parent_avgs = (families.column('father') + families.column('mother'))/2

In [ ]:
heights = Table().with_columns(
    'Parent Average', parent_avgs,
    'Child', families.column('child'),
    'Sex', families.column('sex')
)
heights

In [ ]:
heights.scatter('Parent Average', 'Child')

In [ ]:
heights.scatter('Parent Average', 'Child')
plots.plot([67.5, 67.5], [50, 85], color='red', lw=2)
plots.plot([68.5, 68.5], [50, 85], color='red', lw=2);

In [ ]:
nearby = heights.where('Parent Average', are.between(67.5, 68.5))
nearby_mean = np.average(nearby.column('Child'))
nearby_mean

In [ ]:
heights.scatter('Parent Average', 'Child')
plots.plot([67.5, 67.5], [50, 85], color='red', lw=2)
plots.plot([68.5, 68.5], [50, 85], color='red', lw=2)
plots.scatter(68, nearby_mean, color='red', s=50);

In [ ]:
def predict(h):
    nearby = heights.where('Parent Average', are.between(h - 1/2, h + 1/2))
    return np.average(nearby.column('Child'))

In [ ]:
predict(68)

In [ ]:
predict(70)

In [ ]:
predict(73)

In [ ]:
predicted_heights = heights.apply(predict, 'Parent Average')

In [ ]:
heights = heights.with_column('Prediction', predicted_heights)
heights

In [ ]:
heights.select('Parent Average', 'Child', 'Prediction').scatter('Parent Average')

## Prediction Accuracy ##

In [ ]:
def difference(x, y):
    return x - y

In [ ]:
pred_errs = heights.apply(difference, 'Prediction', 'Child')
heights = heights.with_column('errors',pred_errs)
heights

In [ ]:
heights.hist('errors')

In [ ]:
heights.hist('errors', bins=40)

In [ ]:
heights.hist('errors', group='Sex')

# Improving the model

In [ ]:
def predict_smarter(h, s):
    nearby = heights.where('Parent Average', are.between(h - 1/2, h + 1/2))
    nearby_same_sex = nearby.where('Sex', s)
    return np.average(nearby_same_sex.column('Child'))

In [ ]:
predict_smarter(68, 'female')

In [ ]:
predict_smarter(68, 'male')

In [ ]:
smarter_predicted_heights = heights.apply(predict_smarter, 'Parent Average', 'Sex')
heights = heights.with_column('Smarter Prediction', smarter_predicted_heights)

In [ ]:
smarter_pred_errs = heights.apply(difference, 'Child', 'Smarter Prediction')
heights = heights.with_column('Smarter Errors', smarter_pred_errs)

In [ ]:
heights.hist('Smarter Errors', group='Sex')

In [ ]:
heights.hist('errors', 'Smarter Errors', bins=20)